#Instalação 

In [ ]:
# reading daaset from Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install emoji
!pip install simplejson
!pip install wmd
!pip install kneed

#BTM
!pip install bitermplus
!pip install git+https://github.com/maximtrp/bitermplus.git
!pip install tmplot
!pip install git+https://github.com/maximtrp/tmplot.git

#Metrics
!pip install octis

In [ ]:
 # Packages to store and manipulate data
import pandas as pd
import numpy as np
# Example function using numpy:
from numpy import dot
from numpy.linalg import norm
import math
import os
import json
import simplejson as json
import csv
import string
import glob
import random
import time
import math
from datetime import datetime
pd.set_option('max_colwidth', 200)
 
# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import pickle
import math
import wmd
from scipy import spatial
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
from collections import Counter
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import tmplot as tmp
from tmplot._helpers import get_phi
from tmplot._helpers import calc_terms_probs_ratio

# Packages
import emoji
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import operator
from operator import itemgetter
from tqdm import tqdm
from collections import defaultdict
from textblob import TextBlob
import sys
import itertools
import spacy
from scipy.sparse import csr_matrix, issparse  # , todense
import sys
from collections import defaultdict
import multiprocessing
from pprint import pprint
 
# Gensim
import gensim
import gensim.downloader
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim import models
from gensim import corpora
from gensim.models import CoherenceModel
import gensim.downloader as api
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.models.ldamulticore import LdaMulticore
 
# Model building package: Sklearn
import sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
# Cluster documents: Sklearn
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
 
#BTM
import bitermplus as btm
from typing import List, Union, Tuple, Dict, Sequence, Any
from scipy.sparse import csr
from pandas import DataFrame, Series, concat
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from bitermplus._btm import BTM
 
#PTM
import tomotopy as tp

#Metrics
from octis.evaluation_metrics.metrics import AbstractMetric
from octis.dataset.dataset import Dataset
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import octis.configuration.citations as citations
import numpy as np
import itertools
from scipy import spatial
from sklearn.metrics import pairwise_distances
from operator import add
from octis.evaluation_metrics.rbo import rbo
import gensim.downloader as api
from gensim.models import KeyedVectors
from octis.evaluation_metrics.diversity_metrics import WordEmbeddingsInvertedRBO, \
    WordEmbeddingsInvertedRBOCentroid, InvertedRBO
from itertools import combinations
from scipy.spatial.distance import cosine

#Importar Dados

In [ ]:
df = pd.read_json('/content/drive/dados_fevereiro_dia_de_semana_1.json')

#Tokenizar
new_tweet = []
for tweet in df.clean_tweet:
    tweet_len = [word for word in tweet.split(' ') if len(word) >= 3]
    tweet_string = ' '.join(tweet_len)
    new_tweet.append(tweet_string)
df['clean_tweet'] = new_tweet

stop = stopwords.words('english')
df['clean_tweet_tokenize'] = df['clean_tweet'].apply(lambda x: [item for item in str(x).split(' ')])
df.dropna()

#Modelagem de Tópicos

In [ ]:
def get_words_freqs(
        docs: Union[List[str], np.ndarray, Series],
        **kwargs: dict) -> Tuple[csr.csr_matrix, np.ndarray, Dict]:
    """Compute words vs documents frequency matrix.
    Parameters
    ----------
    docs : Union[List[str], np.ndarray, Series]
        Documents in any format that can be passed to
        :meth:`sklearn.feature_extraction.text.CountVectorizer` method.
    kwargs : dict
        Keyword arguments for
        :meth:`sklearn.feature_extraction.text.CountVectorizer` method.
    Returns
    -------
    Tuple[csr.csr_matrix, np.ndarray, Dict]
        Documents vs words matrix in CSR format,
        vocabulary as a numpy.ndarray of terms,
        and vocabulary as a dictionary of {term: id} pairs.
    Example
    -------
    >>> import pandas as pd
    >>> import bitermplus as btm
    >>> # Loading data
    >>> df = pd.read_csv(
    ...     'dataset/SearchSnippets.txt.gz', header=None, names=['texts'])
    >>> texts = df['texts'].str.strip().tolist()
    >>> # Vectorizing documents, obtaining full vocabulary and biterms
    >>> X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
    """
    vec = CountVectorizer(**kwargs)
    X = vec.fit_transform(docs)
    words = np.array(vec.get_feature_names())
    return X, words, vec.vocabulary_


In [ ]:
# Importing data
texts = df['clean_tweet'].str.strip().tolist()

# PREPROCESSING
# Obtaining terms frequency in a sparse matrix and corpus vocabulary
X, vocabulary, vocab_dict = get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()

# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))

# Generating biterms
biterms = btm.get_biterms(docs_vec)

# INITIALIZING AND RUNNING MODEL
model = btm.BTM(X, vocabulary, T=10)
model.fit(biterms, iterations=100)
p_zd = model.transform(docs_vec)

In [ ]:
import pickle as pkl
# Saving
with open("model.pkl", "wb") as file:
    pkl.dump(model, file)

# Loading
with open("model.pkl", "rb") as file:
    model_train = pkl.load(file)

#Tópicos

In [ ]:
top_words = btm.get_top_topic_words(
    model,
    words_num=10,)
top_words

In [ ]:
topics = []
for i in range(10):
  topics.append(top_words['topic'+str(i)].tolist())

# Visualização

In [ ]:
#Run the interactive report interface
tmp.report(model=model, docs=texts)

In [ ]:
top_docs = btm.get_top_topic_docs(
    texts,
    p_zd,
    docs_num=10,
    )
top_docs

# Métricas

In [ ]:
# cast tweets to numpy array
docs = df.clean_tweet_tokenize.to_numpy()

# create dictionary of all words in all documents
dictionary =  corpora.Dictionary(docs)

# create variable containing length of dictionary/vocab
vocab_length = len(dictionary)

# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
cm_btm = CoherenceModel(topics=topics, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs, 
                          coherence='c_v')

print(cm_btm.get_coherence())

In [ ]:
npmi = CoherenceModel(topics=topics, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs,
                          coherence='c_npmi')

print(npmi.get_coherence())

In [ ]:
def get_word2index(list1, list2):
    words = set(list1)
    words = words.union(set(list2))
    word2index = {w: i for i, w in enumerate(words)}
    return word2index

In [ ]:
#Diversity Metrics:
class TopicDiversity(AbstractMetric):
    def __init__(self, topk=10):
        """
        Initialize metric
        Parameters
        ----------
        topk: top k words on which the topic diversity will be computed
        """
        AbstractMetric.__init__(self)
        self.topk = topk

    def score(self, model_output):
        """
        Retrieves the score of the metric
        Parameters
        ----------
        model_output : dictionary, output of the model
                       key 'topics' required.
        Returns
        -------
        td : score
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than ' + str(self.topk))
        else:
            unique_words = set()
            for topic in topics:
                unique_words = unique_words.union(set(topic[:self.topk]))
            td = len(unique_words) / (self.topk * len(topics))
            return td

In [ ]:
diversity = TopicDiversity()
diversity.score(topics)

In [ ]:
class InvertedRBO(AbstractMetric):
    def __init__(self, topk=10, weight=0.9):
        """
        Initialize metric Inverted Ranked-Biased Overlap
        :param topk: top k words on which the topic diversity will be computed
        :param weight: weight of each agreement at depth d. When set to 1.0, there is no weight, the rbo returns to
        average overlap. (default 0.9)
        """
        super().__init__()
        self.topk = topk
        self.weight = weight

    def score(self, model_output):
        """
        Retrieves the score of the metric
        :param model_output : dictionary, output of the model. the 'topics' key is required.
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')
        else:
            collect = []
            for list1, list2 in itertools.combinations(topics, 2):
                word2index = get_word2index(list1, list2)
                indexed_list1 = [word2index[word] for word in list1]
                indexed_list2 = [word2index[word] for word in list2]
                rbo_val = rbo(indexed_list1[:self.topk], indexed_list2[:self.topk], p=self.weight)[2]
                collect.append(rbo_val)
            return 1 - np.mean(collect)

In [ ]:
b = InvertedRBO()
b.score(topics)